In [1]:
import numpy as np
import pandas as pd
import time
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss


/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/tensorflow/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train_df = pd.read_csv("transformed_train_data.csv")


In [3]:
test_df = pd.read_csv("transformed_test_data.csv")

In [5]:
#Removing outlier
train_df = train_df[train_df['target']>-30]
n_train = len(train_df)
target = train_df["target"]
train_df = train_df.drop(["target"], axis = 1)

In [29]:
features = [c for c in train_df.columns if c not in ['card_id', 'first_active_month']]

In [30]:
param = {'objective':'regression',
         'num_leaves': 31,
         'min_data_in_leaf': 25,
         'max_depth': 7,
         'learning_rate': 0.01,
         'lambda_l1':0.13,
         "boosting": "gbdt",
         "feature_fraction":0.85,
         'bagging_freq':8,
         "bagging_fraction": 0.9 ,
         "metric": 'rmse',
         "verbosity": -1,
         "random_state": 2333}


In [31]:
import time
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
def run_lgbm_cv():
    categorical_feats = ["feature_2", "feature_3"]
    folds = KFold(n_splits = 5, shuffle=True, random_state=15)
    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))
    start = time.time()
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train_df.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=categorical_feats
                              )
        val_data = lgb.Dataset(train_df.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=categorical_feats
                              )

        num_round = 10000
        clf = lgb.train(param,
                        trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        early_stopping_rounds = 200)

        oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits

    print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))
    return feature_importance_df, predictions

In [ ]:
feature_importance_df, predictions = run_lgbm_cv()

fold n°0


/anaconda/envs/tensorflow/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/anaconda/envs/tensorflow/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.60793	valid_1's rmse: 1.62065
[200]	training's rmse: 1.57752	valid_1's rmse: 1.5957
[300]	training's rmse: 1.56167	valid_1's rmse: 1.58427
[400]	training's rmse: 1.55123	valid_1's rmse: 1.57814
[500]	training's rmse: 1.54327	valid_1's rmse: 1.57454
[600]	training's rmse: 1.53623	valid_1's rmse: 1.5721
[700]	training's rmse: 1.53034	valid_1's rmse: 1.57052
[800]	training's rmse: 1.52506	valid_1's rmse: 1.5694
[900]	training's rmse: 1.5202	valid_1's rmse: 1.56872
[1000]	training's rmse: 1.51543	valid_1's rmse: 1.56817
[1100]	training's rmse: 1.5109	valid_1's rmse: 1.56795
[1200]	training's rmse: 1.50666	valid_1's rmse: 1.56754
[1300]	training's rmse: 1.50263	valid_1's rmse: 1.5673
[1400]	training's rmse: 1.49878	valid_1's rmse: 1.56713
[1500]	training's rmse: 1.49483	valid_1's rmse: 1.5671
[1600]	training's rmse: 1.49085	valid_1's rmse: 1.567
[1700]	training's rmse: 1.48701	valid_1's rmse: 1.56689
[18